In [1]:
import os
if os.name != 'nt' :
    import resource
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from App.common import V, init_globals, loading_bar, myfigures, get_query_list, mp, tabs_out, tabs_html
from App import tab1, tab2, tab3, tab4, tab5, tab6
import dmyplant2
from dmyplant2 import cred, MyPlant, Engine, cplotdef, get_size
import logging
import shutil

logfilename = 'FSM.log'
if os.path.exists(logfilename):
    shutil.copyfile(logfilename,logfilename[:-4] + '_bak.log')

logging.basicConfig(
    filename=logfilename,
    filemode='w',
    format='%(levelname)s %(message)s',
    level=logging.DEBUG
)
logging.info("FSM looging Start")

### Statemachine Engine Analysis ©2022 Dieter Chvatal

In [2]:
#########################################
# tabs
#########################################
class App():
    def __init__(self):
        
        self.tab_objects_list = [
            tab1.Tab(),
            tab2.Tab(),
            tab3.Tab(),
            tab4.Tab(),
            tab5.Tab(),
            tab6.Tab()
        ]

        self.tabs = widgets.Tab()
        self.tabs.observe(self.tabs_change_index, 'selected_index')

        self.set_children()
        self.set_titles()

        self.tab_objects_list[0].selected() # preselect the first tab.

    def show(self):
        display(
            widgets.VBox([
                self.tabs,
                tabs_out,
                tabs_html])
            );

    def set_children(self):
        self.tabs.children = [t.tab for t in self.tab_objects_list]

    def set_titles(self):
        for i,t in enumerate(self.tab_objects_list):
            self.tabs.set_title(i,t.title)

    def tabs_change_index(self, *args):  # the tabs callback
        self.tab_objects_list[self.tabs.selected_index].selected()


app = App()
app.show()

In [3]:
if V.fsm:
    pp(V.fsm.results['run2_content'])

In [4]:
if V.fsm:
    df = V.fsm.starts[['alarms','warnings','A','W']]
    dfw = df[df['W']>1]
    [w['msg']['name'] for warnings in  df['warnings'] for w in warnings]

In [5]:
if V.fsm:
    [w['msg']['name'] == '2688' for warnings in  df['warnings'] for w in warnings]

In [6]:
if V.fsm:
    [w['msg']['name'] == '1105' for alarms in  df['alarms'] for w in alarms]

In [7]:
#hasattr(V.fsm, '_messages')

In [8]:
#if V.fsm is not None:
#    V.fsm.act_run, len(V.fsm.starts)

In [9]:
# from App import tab4
# if V.fsm is not None:
#     tab4.update_fig(tab4.sno.value, plotselection=tab4.plotselection.value,plot_range=tab4.time_range.value,fsm=V.fsm, VSC=True)

In [10]:
from pprint import pprint as pp

if os.name != 'nt':
    pass
    #memory = resource.getrusage(resource.RUSAGE_SELF)
    #print(f"Script Memory Usage: {memory.ru_utime:5.1f} MB")

if V.fsm is not None:
    print("\nV.fsm:")
    print("------")
    print(f"{'Size fsm in MB':>40}: {get_size(V.fsm) / (1024.0*1024.0):10.3f} MB")
    keys = list (V.fsm.__dict__.keys())
    for k in keys:
        print(f"{k:>40}: {get_size(V.fsm.__dict__[k]) / (1024.0*1024.0):10.3f} MB")

    print("\n_e:")
    print("--")
    print(f"{'Size fsm._e in MB':>40}: {get_size(V.fsm._e) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm._e.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm._e.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")    

    print("\nstartstopHandler:")
    print("-----------------")
    print(f"{'Size fsm.startstopHandler in MB':>40}: {get_size(V.fsm.startstopHandler) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")

In [11]:
if V.fsm is not None:
    print("\nstartstopHandler._states:")
    print("-------------------------")
    print(f"{'Size fsm.startstopHandler._states in MB':>40}: {get_size(V.fsm.startstopHandler._states) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler._states)
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler._states[sshk]) / (1024.0*1024.0):10.3f} MB")

In [12]:
#V.fsm.save_results('test.dfsm')

In [13]:
#V.fsm._messages

In [14]:
#pd.DataFrame(V.fsm.message_queue[935:937])

In [15]:
#d = V.fsm.message_queue[936:937][0]
#d

In [16]:
#pd.DataFrame([d])

In [17]:
#V.fsm.message_queue[935]

In [18]:
#d=V.fsm.message_queue[936]
#d['assetId'] = 114517
#d

In [19]:
#pd.DataFrame(d, index=[0], dtype=object).T

In [20]:
if V.fsm is not None:
    V.fsm._e._calc_BMEP(4881.0, 1500.0)